## Selenium 사용 동적 크롤링 - 네이버 환율

In [1]:
# Selenium 설치.
!pip install selenium

In [2]:
import pandas as pd
import bs4
from selenium import webdriver
import os   #이부분 새로 작성    #cromedriver.exe새로 깔았음

In [6]:
os.chdir(r"C:\Users\Gram\Desktop\myPyCode\02  데이터 수집과 처리 - 실습") 

### 1. 사이트에 들어가서 탐색해 본다:

1. 먼저 다음 사이트에 들어가서 살펴본다:  [네이버 금융](https://finance.naver.com/marketindex/)
2. 크롬에서 3점 메뉴 버튼을 누르고 **도구 더보기** ==> **개발자 도구**를 선택해서 개발자 도구를 연다.
3. 개발자 도구 왼쪽 상단의 화살표 버튼을 눌러서 사이트 일부를 서택하고 해당 HTML코드를 볼 수 있다. "[CTRL] + [SHIFT] + C" 조합으로 대체할 수 있다.

### 2. 이제는 Selenium을 사용해 본다: 접속해서 데이터 가져와 파싱까지 해줌

In [7]:
# 웹드라이버 로딩.
my_driver = webdriver.Chrome('chromedriver.exe')   #크롬웹드라이버 파일 읽어와야함

In [8]:
# 사이트에 접속.
my_driver.get("https://finance.naver.com/marketindex")   #빈 브라우저가 새로 뜸

In [9]:
# CSS 셀렉터를 사용하여 요소 가져오기. 
res = my_driver.find_elements_by_css_selector('span.value')   #.get해서 네이버금융에 접속  #elements복수니 요소들

In [10]:
# 속성을 직접 가져온다. BeautifulSoup 불필요.
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}   # res로 가져와 정리해서 데이트프레임
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.00
1,JPY,1052.73
2,EUR,1288.11
3,CNY,168.85


### 3. Parsing 부분은 BeautifulSoup로 처리하면 간편하다:

In [11]:
my_html = my_driver.page_source                      # 사이트 내용을 가져온다.
soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용. 

In [12]:
rate =[]
for i in range(4):
    rate += [float(''.join((res[i].text).split(',')))]     # 1000 단위 쉼표 "," 제거 처리.
result = {"외환":["USD","JPY","EUR","CNY"], "환율":rate}
df=pd.DataFrame(result)
df

,외환,환율
0,USD,1159.00
1,JPY,1052.73
2,EUR,1288.11
3,CNY,168.85


### 4. iframe안의 내용 가져오기: 표는 iframe안에 있어 가져올 수 없음(BeautifulSoup로 불가)

In [13]:
# CSS 셀렉터를 사용하여 요소 가져오기. 
res = my_driver.find_elements_by_tag_name('iframe')  #모든 iframe을 다 가져와 elements임

In [14]:
my_html = ''
for x in res:    #x가 iframe객체임
    try:
        my_driver.switch_to_frame(x)    #iframe으로 가기 
                                        #switch_to_frame들어가서 페이지소스 내용가져오고, switch_to_default_content로 나옴
                                                         #iframe하나하나마다 switch로 들어갔다 나왔다 반복       
        my_html += my_driver.page_source + "\n"         # 내용을 이어 붙인다.
        my_driver.switch_to_default_content()           # 복귀
    except:                                             #오류생기면 나오게끔
        my_driver.switch_to_default_content()           # 오류가 나도 복귀

C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:4: DeprecationWarning: use driver.switch_to.frame instead
  after removing the cwd from sys.path.
C:\Anaconda3\lib\site-packages\ipykernel_launcher.py:8: DeprecationWarning: use driver.switch_to.default_content instead
  


In [15]:
# 수집된 iframe 안의 html을 BeautifulSoup 객체로 변환.
soup = bs4.BeautifulSoup(my_html, 'html.parser')     # BeautifulSoup 객체 반환. 'html.parser' 사용.  #모은걸 이제 파싱

In [17]:
res1=soup.find('table')   #표 테이블 찾고
res2=res1.find_all('tr')  #테이블안에서 행을 구분짓기 위해 tr

In [18]:
n = len(res2)            #res2원소 하나하나
for i in range(2,n):
    print(res2[i].text.strip())
    print('-'*50 + '\n')

미국 USD
				
				
1,159.00
1,179.28
1,138.72
1,170.30
1,147.70
1.000
--------------------------------------------------

유럽연합 EUR
				
				
1,288.11
1,313.74
1,262.48
1,300.99
1,275.23
1.111
--------------------------------------------------

일본 JPY (100엔)
					
					
					
				
				
1,052.73
1,071.15
1,034.31
1,063.04
1,042.42
0.908
--------------------------------------------------

중국 CNY
				
				
168.85
177.29
160.41
170.53
167.17
0.146
--------------------------------------------------

홍콩 HKD
				
				
149.19
152.12
146.26
150.68
147.70
0.129
--------------------------------------------------

대만 TWD
				
				
38.69
43.75
35.99
0.00
0.00
0.033
--------------------------------------------------

영국 GBP
				
				
1,511.74
1,541.52
1,481.96
1,526.85
1,496.63
1.304
--------------------------------------------------

오만 OMR
				
				
3,010.23
3,278.14
2,829.62
0.00
0.00
2.597
--------------------------------------------------

캐나다 CAD
				
				
888.53
906.03
871.03
897.41
879.65
0.767

### 5. click으로 페이지 작동:

In [19]:
# 탭 1 선택. 
res = my_driver.find_element_by_css_selector('span.tab1')
res.click()
print(my_driver.current_url)

https://finance.naver.com/marketindex/?tabSel=exchange#tab_section


In [20]:
# 탭 2 선택. 
res = my_driver.find_element_by_css_selector('span.tab2')
res.click()
print(my_driver.current_url)          #같은 사이트에서 새로한 결과 클릭해도 좋음

https://finance.naver.com/marketindex/?tabSel=exchange#tab_section


In [21]:
# 탭 3 선택.  
res = my_driver.find_element_by_css_selector('span.tab3')
res.click()
print(my_driver.current_url)     

https://finance.naver.com/marketindex/?tabSel=gold#tab_section


In [22]:
# 탭 4 선택.  
res = my_driver.find_element_by_css_selector('span.tab4')
res.click()
print(my_driver.current_url)

https://finance.naver.com/marketindex/?tabSel=materials#tab_section
